In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import numpy
import os
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

### 1. Load Test and Train Files

In [3]:
files_train = 0
files_validation = 0

cwd = os.getcwd()
folder = '/content/drive/MyDrive/parking_spots_detector/train_data/train'
for sub_folder in os.listdir(folder):
    path, dirs, files = next(os.walk(os.path.join(folder,sub_folder)))
    files_train += len(files)


folder = '/content/drive/MyDrive/parking_spots_detector/train_data/test'
for sub_folder in os.listdir(folder):
    path, dirs, files = next(os.walk(os.path.join(folder,sub_folder)))
    files_validation += len(files)

print(files_train,files_validation)

381 164


### 2. Set key parameters

In [4]:
img_width, img_height = 128,128
train_data_dir = "/content/drive/MyDrive/parking_spots_detector/train_data/train"
validation_data_dir = "/content/drive/MyDrive/parking_spots_detector/train_data/test"
nb_train_samples = files_train
nb_validation_samples = files_validation
batch_size = 32
epochs = 5
num_classes = 2

### 3. Build model on top of a trained VGG

In [5]:
model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

for layer in model.layers[:10]:
    layer.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [6]:
import tensorflow.keras as kera
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization

X=Flatten()(model.output)
X=Dense(units=2,activation='softmax')(X)
# X = BatchNormalization()(X)  # Add BatchNormalization layer

#  creating our model
model=Model(model.input,X)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [7]:
# Initiate the train and test generators with data Augumentation
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.1,
width_shift_range = 0.1,
height_shift_range=0.1,
rotation_range=5)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.1,
width_shift_range = 0.1,
height_shift_range=0.1,
rotation_range=5)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size,
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

Found 381 images belonging to 2 classes.
Found 164 images belonging to 2 classes.


In [8]:
# Save the model according to the conditions
checkpoint = ModelCheckpoint("car1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


In [9]:
import keras
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])

In [10]:
### Start training!

history_object = model.fit_generator(
train_generator,
epochs = epochs,
validation_data = validation_generator,
callbacks = [checkpoint, early])

<ipython-input-10-79c4311a7792>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_object = model.fit_generator(


Epoch 1/5
12/12 [==============================] - ETA: 0s - loss: 0.3251 - accuracy: 0.8294 

12/12 [==============================] - 217s 18s/step - loss: 0.3251 - accuracy: 0.8294 - val_loss: 0.2291 - val_accuracy: 0.9329
Epoch 2/5
12/12 [==============================] - ETA: 0s - loss: 0.1245 - accuracy: 0.9580 

12/12 [==============================] - 196s 17s/step - loss: 0.1245 - accuracy: 0.9580 - val_loss: 0.1761 - val_accuracy: 0.9329
Epoch 3/5
12/12 [==============================] - ETA: 0s - loss: 0.0551 - accuracy: 0.9843 

12/12 [==============================] - 194s 17s/step - loss: 0.0551 - accuracy: 0.9843 - val_loss: 0.1598 - val_accuracy: 0.9512
Epoch 4/5
12/12 [==============================] - ETA: 0s - loss: 0.0523 - accuracy: 0.9816 

12/12 [==============================] - 197s 17s/step - loss: 0.0523 - accuracy: 0.9816 - val_loss: 0.0721 - val_accuracy: 0.9634
Epoch 5/5
12/12 [==============================] - ETA: 0s - loss: 0.0651 - accuracy: 0.9738 

12/12 [==============================] - 197s 17s/step - loss: 0.0651 - accuracy: 0.9738 - val_loss: 0.1157 - val_accuracy: 0.9512


In [11]:
ref = dict(zip(list(train_generator.class_indices.values()),list(train_generator.class_indices.keys())))


In [12]:
from keras.applications.vgg19 import VGG19,preprocess_input,decode_predictions

from tensorflow.keras.utils import img_to_array,load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [13]:
def prediction(path):
  img=load_img(path,target_size=(128,128))
  i=img_to_array(img)
  im=preprocess_input(i)
  # print(im.shape)
  img=np.expand_dims(im,axis=0)
  # print(img.shape)
  pred=np.argmax(model.predict(img))
  # print(pred)
  print(f"The image belongs to {ref[pred]}")

In [14]:
path="/content/drive/MyDrive/parking_spots_detector/test_images/scene1380.jpg"

prediction(path)

1/1 [==============================] - 0s 402ms/step
The image belongs to occupied
